# 快速入门 GPT-4 Vison

从历史上看，语言模型系统仅接受**文本**作为输入。但是单一的输入形式，限制了大模型的应用落地范围。

随着技术发展，OpenAI 开发的 GPT-4 Turbo with Vision（简称 GPT-4V）允许模型接收**图像**作为输入，并回答关于它们的问题。

📢注意，目前在 Assistants API 中使用 GPT-4 时还不支持图像输入。

## 使用 GPT-4V 识别线上图像（URL）

![image_sample](https://upload.wikimedia.org/wikipedia/commons/thumb/d/dd/Gfp-wisconsin-madison-the-nature-boardwalk.jpg/2560px-Gfp-wisconsin-madison-the-nature-boardwalk.jpg)

In [5]:
from openai import OpenAI
import os

client = OpenAI(base_url=os.getenv("OPENAI_BASE_URL"))

response = client.chat.completions.create(
  model="gpt-4-turbo",
  messages=[
    {
      "role": "user",
      "content": [
        {"type": "text", "text": "介绍下这幅图?"},
        {
          "type": "image_url",
          "image_url": {
            "url": "https://upload.wikimedia.org/wikipedia/commons/thumb/d/dd/Gfp-wisconsin-madison-the-nature-boardwalk.jpg/2560px-Gfp-wisconsin-madison-the-nature-boardwalk.jpg",
          },
        },
      ],
    }
  ],
  max_tokens=300,
)

print(response.choices[0])

Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='这幅图展示了一条木制的栈道穿越一个茂密的草地。天空晴朗，云朵稀疏，给人一种清新宁静的感觉。草地上的草呈现出鲜艳的绿色，远处可以看到少量的树木和丛林。这种环境可能是一个自然保护区或者公园的一部分，是进行散步或自然观察的理想场所。栈道的存在可能是为了保护脆弱的自然地带，同时也提供了一条安全的步行路径。整体上，这是一幅展示大自然美景的图像，带有强烈的宁静与和谐感。', role='assistant', function_call=None, tool_calls=None))


In [6]:
response.choices[0].message.content

'这幅图展示了一条木制的栈道穿越一个茂密的草地。天空晴朗，云朵稀疏，给人一种清新宁静的感觉。草地上的草呈现出鲜艳的绿色，远处可以看到少量的树木和丛林。这种环境可能是一个自然保护区或者公园的一部分，是进行散步或自然观察的理想场所。栈道的存在可能是为了保护脆弱的自然地带，同时也提供了一条安全的步行路径。整体上，这是一幅展示大自然美景的图像，带有强烈的宁静与和谐感。'

### 封装成一个函数 query_image_description

In [7]:
def query_image_description(url, prompt="介绍下这幅图?"):
    client = OpenAI(base_url=os.getenv("OPENAI_BASE_URL"))  # 初始化 OpenAI 客户端
    
    # 发送请求给 OpenAI 的聊天模型
    response = client.chat.completions.create(
        model="gpt-4-turbo",  # 指定使用的模型
        messages=[
            {
                "role": "user",
                "content": [
                    {"type": "text", "text": prompt},
                    {"type": "image_url", "image_url": {"url": url}},
                ],
            }
        ],
        max_tokens=300,
    )
    
    # 返回模型的响应
    return response.choices[0].message.content


### 调用函数测试

![meme_0](https://p6.itc.cn/q_70/images03/20200602/0c267a0d3d814c9783659eb956969ba1.jpeg)

In [8]:
image_url = "https://p6.itc.cn/q_70/images03/20200602/0c267a0d3d814c9783659eb956969ba1.jpeg"
content = query_image_description(image_url)
print(content)

这幅图是一张幽默的对比图，主要展示了两种不同的狗狗形象。左边的狗被赋予了一种极为夸张的肌肉人的形态，头部为柴犬，整个身体类似于健身运动员。图中标有“16岁的我”和“我浑身一片肌肉，每天杠铃，人比较自恋的大胖”，暗示这是年轻时过分理想化的自我形象。

右边的狗则是一只普通坐着的柴犬，体态略显圆润。图中标有“工作后的我”和“好象胖了，好想躺着啥也不管，我不怎么待见走路很慢的人，我爱吃，我懒，家里满小毛”，代表工作后可能因压力和生活方式改变，变得更加朴实、悠闲，甚至有些自嘲


### 使用 GPT-4V 识别本地图像文件（Base64编码）


In [9]:
from openai import OpenAI
import base64
import requests
import os
import json

client = OpenAI()  # 初始化 OpenAI 客户端

def query_base64_image_description(image_path, prompt="解释下图里的内容？", max_tokens=1000):

    # 实现 Base64 编码
    def encode_image(path):
        with open(path, "rb") as image_file:
            return base64.b64encode(image_file.read()).decode('utf-8')

    # 获取图像的 Base64 编码字符串
    base64_image = encode_image(image_path)

    # 构造请求的 HTTP Header
    headers = {
        "Content-Type": "application/json",
        "Authorization": f"Bearer {client.api_key}"
    }

    # 构造请求的负载
    payload = {
        "model": "gpt-4-turbo",
        "messages": [
            {
                "role": "user",
                "content": [
                    {"type": "text", "text": prompt},
                    {"type": "image_url", "image_url": {"url": f"data:image/jpeg;base64,{base64_image}"}}
                ]
            }
        ],
        "max_tokens": max_tokens
    }

    # 发送 HTTP 请求
    response = requests.post(os.getenv("OPENAI_BASE_URL")+"/chat/completions", headers=headers, json=payload)

    # 检查响应并提取所需的 content 字段
    if response.status_code == 200:
        response_data = response.json()
        content = response_data['choices'][0]['message']['content']
        return content
    else:
        return f"Error: {response.status_code}, {response.text}"

#### 使用 Assistants API生成的 GDP 40年对比曲线图

![gdp_data](./images/gdp_1980_2020.jpg)

In [10]:
content = query_base64_image_description("./images/gdp_1980_2020.jpg")
print(content)

这张图表显示了1980年至2020年间美国、中国、日本和德国的国内生产总值（GDP）对比。可以看到：

1. 蓝色线代表美国的GDP走势，其从1980年的较低水平稳步上升，至2020年达到高峰，表现出持续而稳定的增长。

2. 红色线代表中国的GDP走势，从1980年的非常低起点开始，其增长速度从1990年代开始显著加快，2000年后增长尤为迅猛，接近或超过了老牌发达国家。

3. 紫色线代表日本的GDP走势，显示了1980年代后期和1990年代初期的增长，达到高峰后出现平稳或轻微下降的状况，尤其是在1990年代末到2000年代初。

4. 绿色线代表德国的GDP走势，整体上比较平稳，呈现逐步上升的趋势，但增长幅度较为有限。

总的来说，此图表揭示了四个国家在过去40年间的经济表现和发展趋势，其中中国的快速崛起尤其引人注目。


#### 使用 GPT-4V 识别手写体笔记

![](./images/handwriting_0.jpg)

In [11]:
content = query_base64_image_description("./images/handwriting_0.jpg")
print(content)

这张图片展示的是一页笔记，主要讨论了自然语言处理中的几种参数调整技术，特别是在Transformer模型中的应用。具体内容包括：

1. **Prompt Tuning（小提示调整）** - 这是一种只调整模型的一小部分参数来适应特定任务的技术。这里的笔记提到通过调整输入向量X的嵌入（Embedding）和令牌（Token）来实现调整。

2. **Prefix Tuning** - 这种方法涉及在Transformer模型的输入前添加一个固定的前缀W，这个前缀通过与输入X相乘并通过Transformer的Encoder/Decoder来影响输出Y。

3. **LoRA（低秩适应技术）** - 这是通过对模型权重进行低秩适应来调整模型的一种方法。笔记说明了通过调整权重矩阵W来引入一个增量ΔW，进而影响输出Y。这里也提到了关于矩阵维度的具体计算和概念。

4. **Q-LoRA（可能是LoRA技术的一个变体或特定应用）** - 提及了LoRA技术的存储效率，比如LoRA模型需要78GB，而被优化后的Q-LoRA只需要48GB。

这些笔记显示了对各种模型调整技术的理解和分析，常用于提升模型在特定任务上的性能或减少所需的计算资源。


#### 在 Jupyter 标准输出中渲染 Markdown 格式内容

In [12]:
from IPython.display import display, Markdown

# 使用 display 和 Markdown 函数显示 Markdown 内容
display(Markdown(content))

这张图片展示的是一页笔记，主要讨论了自然语言处理中的几种参数调整技术，特别是在Transformer模型中的应用。具体内容包括：

1. **Prompt Tuning（小提示调整）** - 这是一种只调整模型的一小部分参数来适应特定任务的技术。这里的笔记提到通过调整输入向量X的嵌入（Embedding）和令牌（Token）来实现调整。

2. **Prefix Tuning** - 这种方法涉及在Transformer模型的输入前添加一个固定的前缀W，这个前缀通过与输入X相乘并通过Transformer的Encoder/Decoder来影响输出Y。

3. **LoRA（低秩适应技术）** - 这是通过对模型权重进行低秩适应来调整模型的一种方法。笔记说明了通过调整权重矩阵W来引入一个增量ΔW，进而影响输出Y。这里也提到了关于矩阵维度的具体计算和概念。

4. **Q-LoRA（可能是LoRA技术的一个变体或特定应用）** - 提及了LoRA技术的存储效率，比如LoRA模型需要78GB，而被优化后的Q-LoRA只需要48GB。

这些笔记显示了对各种模型调整技术的理解和分析，常用于提升模型在特定任务上的性能或减少所需的计算资源。

![](./images/handwriting_1.jpg)

In [13]:
content = query_base64_image_description("./images/handwriting_1.jpg")
display(Markdown(content))

这张图片展示的是一本笔记本的两页，内容涉及深度学习、自然语言处理（NLP）和特定的技术更新或方法。以下是对这张图片中部分内容的解读：

左页内容：
- 头部标注了“转换器网络”（Transformers），此为深度学习中的一种重要模型结构，常用于处理自然语言处理任务。
- 提到了各种更新或方法，例如“PEFT”和“PFET Methods”，这里可能是指针对预训练语言模型的特定微调方法（Fine-Tuning）。
- 下面列举了多种“Prompt Tuning”的策略或技术，比如“Adapter”、"Prefix"、"Prompt"等，这些都是在不完全重新训练模型的情况下，调整或优化模型以适应新任务的技术。
- 很可能这些笔记是关于如何用不同的技术进行语言模型的微调或适应新任务。

右页内容：
- 讨论了“multi-modality Instruction Fine-Tuning LLMs”，指的可能是对大型多模态语言模型进行指导式微调。
- 提到了多种新技术和方法，如“LoRA”，“PETC 2022”，“Llama”等，这些可能是最近提出的用于语言模型和机器学习的新技术或方法。
- 还有关于“Prefix-tuning & Adapters”的进一步讨论，强调了这些技术在大型语言模型中的应用。

总体上，这些笔记显示了作者对现代NLP中某些先进技术和方法的深入研究和理解。这可能是一个研究人员或学生在学习和整理相关知识点的过程。

## Homework: 


### #1

使用 GPT-4V 识别带有手写体文字的本地图像文件，分享结果。


### #2

整合 `query_base64_image_description` 函数和 Markdown 格式渲染方法，使得输出结果更易阅读。

### #1
![](./images/handwriting_2.jpg)

In [15]:
content = query_base64_image_description("./images/handwriting_2.jpg", prompt='只识别图片中的内容并打印出来')
display(content)

'图片中的内容包括：\n\n1. 海伦探望朋友,对老款三人.\n2. All roads lead to Rome.\n3. A×B = {(x,y)|x∈A ∧ y∈B}\n4. 表格：\n   - t1 | 10% | 0.8\n   - t2 | 20% | 1.2\n   - t3 | 30% | 1.2'

In [16]:
content = query_base64_image_description("./images/handwriting_2.jpg", prompt='识别图中内容，转换成markdown格式输出')
display(Markdown(content))

```markdown
1. 金枪鱼腥味明显,对宠物喵三人。
2. All roads lead to Rome.
3. A×B = {(x,y)|x∈A∧y∈B}
4. 表格

|    | t1 | t2 | t3  |
|----|----|----|-----|
| %  | 10%| 20%| 30% |
| 值 | 0.8| 1.2| 1.9.2|
```